In [2]:
import os
import collections
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_addons as tfa
import tensorflow_text as text
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

2021-09-19 07:01:38.608318: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
tf.get_logger().setLevel("ERROR")

In [4]:
!nvidia-smi

Sun Sep 19 07:01:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data prep

In [5]:
root_dir = "../data"
sub_root_dir = os.path.join(root_dir, "gt_processed")
images_dir = os.path.join(sub_root_dir, "images")
tfrecords_dir = os.path.join(sub_root_dir, "tfrecords")
lp_file = os.path.join(sub_root_dir, "correct_data.json")

In [6]:


with open(lp_file, "r") as f:
    annotations = json.load(f)["annotations"]

image_path_to_lp = collections.defaultdict(list)

for element in annotations:
    postocr = f"{element['postocr']}"
    gt = f"{element['gt']}"
    image_path = os.path.join(images_dir, element["image"])
#     image_path_to_lp[image_path].append(postocr)
    image_path_to_lp[image_path].append(gt)


image_paths = list(image_path_to_lp.keys())
print(f"Number of images: {len(image_paths)}")

Number of images: 11400


In [7]:
train_size = 9000
valid_size = len(image_paths) - train_size
lps_per_image = 2
train_images_per_file = 3000
valid_images_per_file = 800

train_image_paths = image_paths[:train_size]
num_train_files = int(np.ceil(train_size / train_images_per_file))
train_files_prefix = os.path.join(tfrecords_dir, "train")

valid_image_paths = image_paths[-valid_size:]
num_valid_files = int(np.ceil(valid_size / valid_images_per_file))
valid_files_prefix = os.path.join(tfrecords_dir, "valid")

In [67]:
tf.io.gfile.makedirs(tfrecords_dir)


def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def create_feature(image_path, lp):
    feature = {
        "lp": bytes_feature(lp.encode()),
        "image": bytes_feature(tf.io.read_file(image_path).numpy()),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def write_tfrecords(file_name, image_paths):
    lp_list = []
    image_path_list = []
    for image_path in image_paths:
        lps = image_path_to_lp[image_path]
        lp_list.extend(lps)
        image_path_list.extend([image_path] * len(lps))
    with tf.io.TFRecordWriter(file_name) as writer:
        for example_idx in range(len(image_path_list)):
            example = create_feature(
                image_path_list[example_idx], lp_list[example_idx]
            )
            writer.write(example.SerializeToString())
    return example_idx + 1


def write_data(image_paths, num_files, files_prefix, images_per_file):
    example_counter = 0
    for file_idx in tqdm(range(num_files)):
        file_name = files_prefix + "-%02d.tfrecord" % (file_idx)
        start_idx = images_per_file * file_idx
        end_idx = start_idx + images_per_file
        example_counter += write_tfrecords(file_name, image_paths[start_idx:end_idx])
    return example_counter


In [68]:
train_example_count = write_data(train_image_paths, num_train_files, train_files_prefix, train_images_per_file)
valid_example_count = write_data(valid_image_paths, num_valid_files, valid_files_prefix, valid_images_per_file)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.34it/s]


# Model

In [8]:
def project_embeddings(
    embeddings, num_projection_layers, projection_dims, dropout_rate
):
    projected_embeddings = layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = layers.Dense(projection_dims)(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Add()([projected_embeddings, x])
        projected_embeddings = layers.LayerNormalization()(x)
    return projected_embeddings

In [9]:
def create_vision_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    xception = keras.applications.Xception(
        include_top=False, weights="imagenet", pooling="avg"
    )
    for layer in xception.layers:
        layer.trainable = trainable
    inputs = layers.Input(shape=(299, 299, 3), name="image_input")
    xception_input = tf.keras.applications.xception.preprocess_input(inputs)
    embeddings = xception(xception_input)
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    return keras.Model(inputs, outputs, name="vision_encoder")

In [29]:
def create_text_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    preprocess = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2",
        name="text_preprocessing",
    )
    bert = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1",
        "bert",
    )
    bert.trainable = trainable
    inputs = layers.Input(shape=(), dtype=tf.string, name="text_input")
    bert_inputs = preprocess(inputs)
    embeddings = bert(bert_inputs)["pooled_output"]
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    return keras.Model(inputs, outputs, name="text_encoder")

In [36]:
class IGTCPreTrainer(keras.Model):
    def __init__(self, text_encoder, image_encoder, **kwargs):
        super(IGTCPreTrainer, self).__init__(**kwargs)
        self.text_encoder = text_encoder
        self.image_encoder = image_encoder
        self.temp = tf.Variable(1.)
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        lp_embeddings = text_encoder(features["lp"], training=training)
        image_embeddings = vision_encoder(features["image"], training=training)
        return lp_embeddings, image_embeddings, tf.math.exp(tf.math.scalar_mul(self.temp, tf.math.log(1/0.07)))

    def compute_loss(self, lp_embeddings, image_embeddings, logit_scale):
        logits = (
            tf.matmul(lp_embeddings, image_embeddings, transpose_b=True)
            * logit_scale
        )
        images_similarity = tf.matmul(
            image_embeddings, image_embeddings, transpose_b=True
        )
        lp_similarity = tf.matmul(
            lp_embeddings, lp_embeddings, transpose_b=True
        )
        targets = keras.activations.softmax(
            (lp_similarity + images_similarity) / 2
        )
        lp_loss = keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        images_loss = keras.losses.categorical_crossentropy(
            y_true=tf.transpose(targets), y_pred=tf.transpose(logits), from_logits=True
        )
        return (lp_loss + images_loss) / 2

    def train_step(self, features):
        with tf.GradientTape() as tape:
            lp_embeddings, image_embeddings, logit_scale = self(features, training=True)
            logit_scale = tf.clip_by_value(logit_scale, clip_value_min=0, clip_value_max=4.6052)
            loss = self.compute_loss(lp_embeddings, image_embeddings, logit_scale)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}
 
    def test_step(self, features):
        lp_embeddings, image_embeddings, logit_scale = self(features, training=False)
        logit_scale = tf.clip_by_value(logit_scale, clip_value_min=0, clip_value_max=4.6052)
        loss = self.compute_loss(lp_embeddings, image_embeddings, logit_scale)
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

In [37]:
num_epochs = 5
batch_size = 128

In [38]:
vision_encoder = create_vision_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.2)
text_encoder = create_text_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1)
pretrainer = IGTCPreTrainer(text_encoder, vision_encoder)
pretrainer.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.001)
)

In [39]:
def decode_fn(sample):
    features = tf.io.parse_single_example(
        sample,
        {
           "lp": tf.io.FixedLenFeature([], dtype=tf.string),
           "image": tf.io.FixedLenFeature([], dtype=tf.string)
        }
    )
    features['image'] = tf.io.decode_jpeg(
        features['image'], channels=3
    )
    return features
    

def fetch_dataset_tfrecord(string_pattern, batch_size, shuffle_size):
    return (
        tf.data.TFRecordDataset(
            tf.data.Dataset.list_files(string_pattern))
        .map(decode_fn, num_parallel_calls=8)
        .shuffle(shuffle_size)
        .batch(batch_size)
    )

In [40]:
train_dataset = fetch_dataset_tfrecord(
    os.path.join(tfrecords_dir, "train-*.tfrecord"),
    batch_size,
    train_size
)
valid_dataset = fetch_dataset_tfrecord(
    os.path.join(tfrecords_dir, "valid-*.tfrecord"),
    batch_size,
    valid_size
)

In [41]:
reduce_learningRate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2, patience=3
)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='clipPretrained_model_{epoch:02d}-{val_loss:.2f}.h5')

history = pretrainer.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=[reduce_learningRate, model_checkpoint]
)

Epoch 1/5
71/71 [==============================] - 105s 1s/step - loss: 31.1796 - val_loss: 10.1511
Epoch 2/5
71/71 [==============================] - 95s 1s/step - loss: 9.9929 - val_loss: 5.3923
Epoch 3/5
71/71 [==============================] - 95s 1s/step - loss: 5.7545 - val_loss: 4.9945
Epoch 4/5
71/71 [==============================] - 95s 1s/step - loss: 5.0514 - val_loss: 4.9133
Epoch 5/5
71/71 [==============================] - 95s 1s/step - loss: 4.9202 - val_loss: 4.8925


In [1]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["train", "valid"], loc="upper right")
plt.show()

NameError: name 'plt' is not defined